In [1]:
import os
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Создание папки для модифицированных промежуточных файлов
intermediate_path_modified = "./multiclass_models_results_modified_max-1--3/"
os.makedirs(intermediate_path_modified, exist_ok=True)

# Путь к промежуточным файлам
intermediate_path = "./multiclass_models_results/"
model_results_files = [f for f in os.listdir(intermediate_path) if f.endswith("_multiclass_results.csv")]

# Создаем итоговый DataFrame для всех моделей
final_results = []

# Процесс обработки каждого промежуточного файла модели
for model_results_file in model_results_files:
    # Чтение промежуточного CSV файла
    df = pd.read_csv(os.path.join(intermediate_path, model_results_file))
    # display(df)

    # Получаем список метрик, исключая "Packet Size" и "Model"
    metrics_columns = [col for col in df.columns if col not in ['Packet Size', 'Model', 'File']]

    # Определение лучшей выборки для каждой метрики
    best = {}
    counts = {}

    # Определяем 1, 2 и 3 максимумы по каждому столбцу в пределах самого столбца
    metrics_cols_max_values = {col: df[col].nlargest(3).values for col in metrics_columns}
    # print(metrics_cols_max_values)

    # Добавление столбцов для первого, второго и третьего максимумов
    max1_counts_column = []
    max2_counts_column = []
    max3_counts_column = []

    for index, row in df.iterrows():
        # Создаем словарь значений метрик для текущей строки

        # Сортируем уникальные значения по убыванию
        # sorted_values = sorted(set(metrics_cols_max_values.values()), reverse=True)

        # Подсчет для первого максимума
        max1_count = sum(
            1 for col in metrics_columns if row[col] == metrics_cols_max_values[col][0]
        )
        max1_counts_column.append(max1_count)

        # Подсчет для второго максимума (если есть)
        # max2_count = 0

        max2_count = sum(
            1 for col in metrics_columns if row[col] == metrics_cols_max_values[col][1]
        )
        max2_counts_column.append(max2_count)

        # Подсчет для третьего максимума (если есть)
        # max3_count = 0

        max3_count = sum(
            1 for col in metrics_columns if row[col] == metrics_cols_max_values[col][2]
        )
        max3_counts_column.append(max3_count)

    df["max 1"] = max1_counts_column
    df["max 2"] = max2_counts_column
    df["max 3"] = max3_counts_column

    # Для каждой метрики находим наилучшие выборки
    for col in metrics_columns:  # Перебираем только метрики
        max_value = df[col].max()  # Находим максимальное значение для метрики
        best[col] = df[df[col] == max_value]["Packet Size"].tolist()  # Сохраняем размеры выборок с максимальным значением

        # Подсчитаем, сколько раз каждая выборка была максимальной для этой метрики
        counts[col] = df[col].value_counts().get(max_value, 0)  # Подсчитаем сколько раз максимальное значение встречается

    # Строка Best
    best_row = {"Packet Size": "Best"}
    for col in metrics_columns:
        best_row[col] = best[col]  # Присваиваем лучшие выборки для каждой метрики

    # Добавление строки Best в DataFrame
    df_best = pd.DataFrame([best_row])

    # Добавление строки Best в DataFrame
    df = pd.concat([df, df_best], ignore_index=True)

    model_results_file_modified = os.path.join(
            intermediate_path_modified, f"{os.path.splitext(model_results_file)[0]}_modified_max-1--3.csv"
        )

    # Запись результатов в новый CSV файл
    df.to_csv(model_results_file_modified, index=False)

{'Accuracy': array([0.847, 0.841, 0.839]), 'Macro F1 Score': array([0.858, 0.825, 0.765]), 'Micro F1 Score': array([0.847, 0.841, 0.839]), 'Weighted F1 Score': array([0.843, 0.841, 0.839]), 'Macro Precision': array([0.885, 0.837, 0.777]), 'Micro Precision': array([0.847, 0.841, 0.839]), "Cohen's Kappa": array([0.81 , 0.804, 0.802]), 'Precision_0': array([0.733, 0.726, 0.69 ]), 'Recall_0': array([0.851, 0.831, 0.827]), 'F1_0': array([0.762, 0.759, 0.747]), 'Precision_1': array([1.   , 1.   , 0.985]), 'Recall_1': array([1.   , 0.949, 0.938]), 'F1_1': array([0.968, 0.957, 0.931]), 'Precision_2': array([1.   , 1.   , 0.987]), 'Recall_2': array([1.   , 1.   , 0.981]), 'F1_2': array([1.   , 0.991, 0.95 ]), 'Precision_3': array([1.   , 1.   , 0.927]), 'Recall_3': array([0.933, 0.875, 0.875]), 'F1_3': array([0.933, 0.93 , 0.889]), 'Precision_4': array([1.   , 0.965, 0.955]), 'Recall_4': array([0.933, 0.9  , 0.875]), 'F1_4': array([0.947, 0.944, 0.877]), 'Precision_5': array([0.967, 0.929, 0.91